In [1]:
import os
import h5torch
import numpy as np
import requests

In [2]:
data_folder = "/data/home/natant/Negatives/Data/Encode690/ENCODE_hg38_subset_101bp_celltypes_ATAC_H5_all_chr"
h5t_files = [f for f in os.listdir(data_folder) if f.endswith('.h5t')]
dicts = {}
for h5t_file in h5t_files:
    file_path = os.path.join(data_folder, h5t_file)
    file = h5torch.File(file_path, 'r')
    prot_names = file["0/prot_names"][:]
    central = file["central"][:]
    celltype = h5t_file.split('.')[0]
    atac_idx = np.where(prot_names == b"ATAC_peak")[0]
    central_no_atac = np.delete(central, atac_idx, axis=0)
    mask_all_zeros = (central_no_atac == 0).all(axis=0)

    filtered_no_ATAC = central_no_atac[:, ~mask_all_zeros]

    # Count 0's and 1's for each row
    zeros_per_row = (filtered_no_ATAC == 0).sum(axis=1)
    ones_per_row = (filtered_no_ATAC == 1).sum(axis=1)

    tf_stats = {}
    for i, name in enumerate(prot_names[1:]):  # skip ATAC_peak
        tf_name = name.decode()
        num_positives = ones_per_row[i]
        num_negatives = zeros_per_row[i]
        tf_stats[tf_name] = (int(num_positives), int(num_negatives))
    dicts[celltype] = tf_stats

In [3]:
dicts

{'MCF-7': {'CTCF': (21001, 36689),
  'TCF7L2': (10297, 405259),
  'ZNF217': (9945, 404761),
  'GATA3_(SC-268)': (6083, 415270)},
 'GM12878': {'CTCF': (40107, 398210),
  'YY1_(SC-281)': (30998, 349533),
  'TBP': (14890, 432228),
  'Egr-1': (16324, 424484),
  'Mxi1_(AF4185)': (17747, 403510),
  'SRF': (8546, 468662),
  'MAZ_(ab85725)': (18972, 395102),
  'ELK1_(1277-1)': (5584, 483172),
  'SIX5': (4843, 489275),
  'USF-1': (9779, 475641),
  'SP1': (18248, 376855),
  'RFX5_(200-401-194)': (4340, 504861),
  'ELF1_(SC-631)': (22998, 374019),
  'ATF2_(SC-81188)': (23490, 372754),
  'NF-YB': (13305, 474014),
  'USF2': (9028, 477902),
  'Znf143_(16618-1-AP)': (20018, 418516),
  'ZEB1_(SC-25388)': (4842, 506221),
  'Pbx3': (9935, 480355),
  'MEF2A': (17612, 408335),
  'TCF12': (20437, 394834),
  'Max': (12553, 438130),
  'STAT5A_(SC-74442)': (7432, 476836),
  'NFIC_(SC-81335)': (29066, 367313),
  'Nrf1': (5689, 496726),
  'CEBPB_(SC-150)': (5798, 484441),
  'FOXM1_(SC-502)': (22935, 363802),
  

In [5]:
# EXERPT FROM DATALOADER CODE!!!!
part1 =  ['chr13', 'chr13_KI270838v1_alt', 'chr13_KI270839v1_alt', 'chr13_KI270840v1_alt', 'chr13_KI270841v1_alt', 'chr13_KI270842v1_alt', 'chr13_KI270843v1_alt', 'chr18', 'chr18_GL383567v1_alt', 'chr18_GL383568v1_alt', 'chr18_GL383569v1_alt', 'chr18_GL383570v1_alt', 'chr18_GL383571v1_alt', 'chr18_GL383572v1_alt', 'chr18_KI270863v1_alt', 'chr18_KI270864v1_alt', 'chr18_KI270911v1_alt', 'chr18_KI270912v1_alt', 'chr19', 'chr19_GL000209v2_alt', 'chr19_GL383573v1_alt', 'chr19_GL383574v1_alt', 'chr19_GL383575v2_alt', 'chr19_GL383576v1_alt', 'chr19_GL949746v1_alt', 'chr19_GL949747v2_alt', 'chr19_GL949748v2_alt', 'chr19_GL949749v2_alt', 'chr19_GL949750v2_alt', 'chr19_GL949751v2_alt', 'chr19_GL949752v1_alt', 'chr19_GL949753v2_alt', 'chr19_KI270865v1_alt', 'chr19_KI270866v1_alt', 'chr19_KI270867v1_alt', 'chr19_KI270868v1_alt', 'chr19_KI270882v1_alt', 'chr19_KI270883v1_alt', 'chr19_KI270884v1_alt', 'chr19_KI270885v1_alt', 'chr19_KI270886v1_alt', 'chr19_KI270887v1_alt', 'chr19_KI270888v1_alt', 'chr19_KI270889v1_alt', 'chr19_KI270890v1_alt', 'chr19_KI270891v1_alt', 'chr19_KI270914v1_alt', 'chr19_KI270915v1_alt', 'chr19_KI270916v1_alt', 'chr19_KI270917v1_alt', 'chr19_KI270918v1_alt', 'chr19_KI270919v1_alt', 'chr19_KI270920v1_alt', 'chr19_KI270921v1_alt', 'chr19_KI270922v1_alt', 'chr19_KI270923v1_alt', 'chr19_KI270929v1_alt', 'chr19_KI270930v1_alt', 'chr19_KI270931v1_alt', 'chr19_KI270932v1_alt', 'chr19_KI270933v1_alt', 'chr19_KI270938v1_alt', 'chr20', 'chr20_GL383577v2_alt', 'chr20_KI270869v1_alt', 'chr20_KI270870v1_alt', 'chr20_KI270871v1_alt', 'chr3', 'chr3_GL000221v1_random', 'chr3_GL383526v1_alt', 'chr3_JH636055v2_alt', 'chr3_KI270777v1_alt', 'chr3_KI270778v1_alt', 'chr3_KI270779v1_alt', 'chr3_KI270780v1_alt', 'chr3_KI270781v1_alt', 'chr3_KI270782v1_alt', 'chr3_KI270783v1_alt', 'chr3_KI270784v1_alt', 'chr3_KI270895v1_alt', 'chr3_KI270924v1_alt', 'chr3_KI270934v1_alt', 'chr3_KI270935v1_alt', 'chr3_KI270936v1_alt', 'chr3_KI270937v1_alt', 'chr4', 'chr4_GL000008v2_random', 'chr4_GL000257v2_alt', 'chr4_GL383527v1_alt', 'chr4_GL383528v1_alt', 'chr4_KI270785v1_alt', 'chr4_KI270786v1_alt', 'chr4_KI270787v1_alt', 'chr4_KI270788v1_alt', 'chr4_KI270789v1_alt', 'chr4_KI270790v1_alt', 'chr4_KI270896v1_alt', 'chr4_KI270925v1_alt', 'chr7', 'chr7_GL383534v2_alt', 'chr7_KI270803v1_alt', 'chr7_KI270804v1_alt', 'chr7_KI270805v1_alt', 'chr7_KI270806v1_alt', 'chr7_KI270807v1_alt', 'chr7_KI270808v1_alt', 'chr7_KI270809v1_alt', 'chr7_KI270899v1_alt', 'chrX', 'chrX_KI270880v1_alt', 'chrX_KI270881v1_alt', 'chrX_KI270913v1_alt']
part2 = ['chr1', 'chr10', 'chr10_GL383545v1_alt', 'chr10_GL383546v1_alt', 'chr10_KI270824v1_alt', 'chr10_KI270825v1_alt', 'chr11', 'chr11_GL383547v1_alt', 'chr11_JH159136v1_alt', 'chr11_JH159137v1_alt', 'chr11_KI270721v1_random', 'chr11_KI270826v1_alt', 'chr11_KI270827v1_alt', 'chr11_KI270829v1_alt', 'chr11_KI270830v1_alt', 'chr11_KI270831v1_alt', 'chr11_KI270832v1_alt', 'chr11_KI270902v1_alt', 'chr11_KI270903v1_alt', 'chr11_KI270927v1_alt', 'chr15', 'chr15_GL383554v1_alt', 'chr15_GL383555v2_alt', 'chr15_KI270727v1_random', 'chr15_KI270848v1_alt', 'chr15_KI270849v1_alt', 'chr15_KI270850v1_alt', 'chr15_KI270851v1_alt', 'chr15_KI270852v1_alt', 'chr15_KI270905v1_alt', 'chr15_KI270906v1_alt', 'chr1_GL383518v1_alt', 'chr1_GL383519v1_alt', 'chr1_GL383520v2_alt', 'chr1_KI270706v1_random', 'chr1_KI270707v1_random', 'chr1_KI270708v1_random', 'chr1_KI270709v1_random', 'chr1_KI270710v1_random', 'chr1_KI270711v1_random', 'chr1_KI270712v1_random', 'chr1_KI270713v1_random', 'chr1_KI270714v1_random', 'chr1_KI270759v1_alt', 'chr1_KI270760v1_alt', 'chr1_KI270761v1_alt', 'chr1_KI270762v1_alt', 'chr1_KI270763v1_alt', 'chr1_KI270764v1_alt', 'chr1_KI270765v1_alt', 'chr1_KI270766v1_alt', 'chr1_KI270892v1_alt', 'chr21', 'chr21_GL383578v2_alt', 'chr21_GL383579v2_alt', 'chr21_GL383580v2_alt', 'chr21_GL383581v2_alt', 'chr21_KI270872v1_alt', 'chr21_KI270873v1_alt', 'chr21_KI270874v1_alt', 'chr22', 'chr22_GL383582v2_alt', 'chr22_GL383583v2_alt', 'chr22_KB663609v1_alt', 'chr22_KI270731v1_random', 'chr22_KI270732v1_random', 'chr22_KI270733v1_random', 'chr22_KI270734v1_random', 'chr22_KI270735v1_random', 'chr22_KI270736v1_random', 'chr22_KI270737v1_random', 'chr22_KI270738v1_random', 'chr22_KI270739v1_random', 'chr22_KI270875v1_alt', 'chr22_KI270876v1_alt', 'chr22_KI270877v1_alt', 'chr22_KI270878v1_alt', 'chr22_KI270879v1_alt', 'chr22_KI270928v1_alt', 'chr9', 'chr9_GL383539v1_alt', 'chr9_GL383540v1_alt', 'chr9_GL383541v1_alt', 'chr9_GL383542v1_alt', 'chr9_KI270717v1_random', 'chr9_KI270718v1_random', 'chr9_KI270719v1_random', 'chr9_KI270720v1_random', 'chr9_KI270823v1_alt', 'chrY', 'chrY_KI270740v1_random']
part3 = ['chr12', 'chr12_GL383549v1_alt', 'chr12_GL383550v2_alt', 'chr12_GL383551v1_alt', 'chr12_GL383552v1_alt', 'chr12_GL383553v2_alt', 'chr12_GL877875v1_alt', 'chr12_GL877876v1_alt', 'chr12_KI270833v1_alt', 'chr12_KI270834v1_alt', 'chr12_KI270835v1_alt', 'chr12_KI270836v1_alt', 'chr12_KI270837v1_alt', 'chr12_KI270904v1_alt', 'chr14', 'chr14_GL000009v2_random', 'chr14_GL000194v1_random', 'chr14_GL000225v1_random', 'chr14_KI270722v1_random', 'chr14_KI270723v1_random', 'chr14_KI270724v1_random', 'chr14_KI270725v1_random', 'chr14_KI270726v1_random', 'chr14_KI270844v1_alt', 'chr14_KI270845v1_alt', 'chr14_KI270846v1_alt', 'chr14_KI270847v1_alt', 'chr16', 'chr16_GL383556v1_alt', 'chr16_GL383557v1_alt', 'chr16_KI270728v1_random', 'chr16_KI270853v1_alt', 'chr16_KI270854v1_alt', 'chr16_KI270855v1_alt', 'chr16_KI270856v1_alt', 'chr17', 'chr17_GL000205v2_random', 'chr17_GL000258v2_alt', 'chr17_GL383563v3_alt', 'chr17_GL383564v2_alt', 'chr17_GL383565v1_alt', 'chr17_GL383566v1_alt', 'chr17_JH159146v1_alt', 'chr17_JH159147v1_alt', 'chr17_JH159148v1_alt', 'chr17_KI270729v1_random', 'chr17_KI270730v1_random', 'chr17_KI270857v1_alt', 'chr17_KI270858v1_alt', 'chr17_KI270859v1_alt', 'chr17_KI270860v1_alt', 'chr17_KI270861v1_alt', 'chr17_KI270862v1_alt', 'chr17_KI270907v1_alt', 'chr17_KI270908v1_alt', 'chr17_KI270909v1_alt', 'chr17_KI270910v1_alt', 'chr2', 'chr2_GL383521v1_alt', 'chr2_GL383522v1_alt', 'chr2_GL582966v2_alt', 'chr2_KI270715v1_random', 'chr2_KI270716v1_random', 'chr2_KI270767v1_alt', 'chr2_KI270768v1_alt', 'chr2_KI270769v1_alt', 'chr2_KI270770v1_alt', 'chr2_KI270771v1_alt', 'chr2_KI270772v1_alt', 'chr2_KI270773v1_alt', 'chr2_KI270774v1_alt', 'chr2_KI270775v1_alt', 'chr2_KI270776v1_alt', 'chr2_KI270893v1_alt', 'chr2_KI270894v1_alt', 'chr5', 'chr5_GL000208v1_random', 'chr5_GL339449v2_alt', 'chr5_GL383530v1_alt', 'chr5_GL383531v1_alt', 'chr5_GL383532v1_alt', 'chr5_GL949742v1_alt', 'chr5_KI270791v1_alt', 'chr5_KI270792v1_alt', 'chr5_KI270793v1_alt', 'chr5_KI270794v1_alt', 'chr5_KI270795v1_alt', 'chr5_KI270796v1_alt', 'chr5_KI270897v1_alt', 'chr5_KI270898v1_alt', 'chr6', 'chr6_GL000250v2_alt', 'chr6_GL000251v2_alt', 'chr6_GL000252v2_alt', 'chr6_GL000253v2_alt', 'chr6_GL000254v2_alt', 'chr6_GL000255v2_alt', 'chr6_GL000256v2_alt', 'chr6_GL383533v1_alt', 'chr6_KB021644v2_alt', 'chr6_KI270758v1_alt', 'chr6_KI270797v1_alt', 'chr6_KI270798v1_alt', 'chr6_KI270799v1_alt', 'chr6_KI270800v1_alt', 'chr6_KI270801v1_alt', 'chr6_KI270802v1_alt', 'chr8', 'chr8_KI270810v1_alt', 'chr8_KI270811v1_alt', 'chr8_KI270812v1_alt', 'chr8_KI270813v1_alt', 'chr8_KI270814v1_alt', 'chr8_KI270815v1_alt', 'chr8_KI270816v1_alt', 'chr8_KI270817v1_alt', 'chr8_KI270818v1_alt', 'chr8_KI270819v1_alt', 'chr8_KI270820v1_alt', 'chr8_KI270821v1_alt', 'chr8_KI270822v1_alt', 'chr8_KI270900v1_alt', 'chr8_KI270901v1_alt', 'chr8_KI270926v1_alt']
parts = [part1, part2, part3]

TF = "CTCF"
subset = part1
prot_mask = file["0/prot_names"][:] == TF.encode()
central = file["central"][prot_mask].squeeze() #only take data for that TF
pos_mask = central == 1 #only take the positive samples

central = central[pos_mask] #subset data

peak_ix_to_pos = file["1/peak_ix_to_pos"][pos_mask]
peak_ix_to_chr = file["1/peak_ix_to_chr"][pos_mask]


pos_subset_mask = np.isin(peak_ix_to_chr.astype(str), subset) #only take the right (training, val) positions
peak_ix_to_pos = peak_ix_to_pos[pos_subset_mask]
peak_ix_to_chr = peak_ix_to_chr[pos_subset_mask]

In [8]:
cv_group_stats_tuple = {}

for h5t_file in h5t_files:
    file_path = os.path.join(data_folder, h5t_file)
    file = h5torch.File(file_path, 'r')
    prot_names = file["0/prot_names"][:]
    celltype = h5t_file.split('.')[0]

    tf_stats = {}
    for i, name in enumerate(prot_names):
        if name == b"ATAC_peak":
            continue
        tf_name = name.decode()
        tf_mask = prot_names == name
        central = file["central"][tf_mask].squeeze()
        pos_mask = central == 1
        peak_ix_to_chr = file["1/peak_ix_to_chr"][:][pos_mask]
        part_counts = tuple(np.isin(peak_ix_to_chr.astype(str), part).sum() for part in parts)
        tf_stats[tf_name] = part_counts
    cv_group_stats_tuple[celltype] = tf_stats

# Example: show for A549
cv_group_stats_tuple['A549']


{'CTCF': (np.int64(13342), np.int64(13624), np.int64(18769)),
 'YY1_(SC-281)': (np.int64(2807), np.int64(2951), np.int64(4519)),
 'CREB1_(SC-240)': (np.int64(4496), np.int64(4598), np.int64(6813)),
 'Max': (np.int64(2668), np.int64(2801), np.int64(4422)),
 'TCF12': (np.int64(5817), np.int64(5829), np.int64(9260)),
 'FOSL2': (np.int64(7796), np.int64(8467), np.int64(12500)),
 'ELF1_(SC-631)': (np.int64(2357), np.int64(2480), np.int64(3781)),
 'BHLHE40': (np.int64(850), np.int64(890), np.int64(1385)),
 'ATF3': (np.int64(1631), np.int64(1876), np.int64(3081)),
 'USF-1': (np.int64(2170), np.int64(2457), np.int64(3821)),
 'ETS1': (np.int64(1535), np.int64(1503), np.int64(2503)),
 'SIX5': (np.int64(1488), np.int64(1318), np.int64(2109)),
 'ZBTB33': (np.int64(1896), np.int64(2039), np.int64(3222)),
 'FOXA1_(SC-101058)': (np.int64(2180), np.int64(2119), np.int64(3401))}

In [10]:
import pickle

with open("/data/home/natant/Negatives/TFBS_negatives/testing_ground/20250613_NUMBER_OF_POSITIVES_PER_GROUP.pkl", "wb") as f:
    pickle.dump(cv_group_stats_tuple, f)

import pickle

with open("/data/home/natant/Negatives/TFBS_negatives/testing_ground/20250603_NUMBER_OF_POSITIVES.pkl", "wb") as f:
    pickle.dump(dicts, f)

In [4]:
import pandas as pd

# Create a table for each celltype
tf_tables = {}
for cell, stats in dicts.items():
    tf_names = list(stats.keys())
    positives = [stats[tf][0] for tf in tf_names]
    negatives = [stats[tf][1] for tf in tf_names]
    tf_tables[cell] = pd.DataFrame({'Positives': positives, 'Negatives': negatives}, index=tf_names)

# Example: display the table for A549
tf_tables['A549']

,Positives,Negatives
CTCF,45735,175687
YY1_(SC-281),10277,234169
CREB1_(SC-240),15907,210554
Max,9891,231598
TCF12,20906,190336
FOSL2,28763,183745
ELF1_(SC-631),8618,232142
BHLHE40,3125,260466
ATF3,6588,237017
USF-1,8448,209389


In [7]:
for celltype, table in tf_tables.items():
    print(f"Celltype: {celltype}")
    print(table)
    print("\n" + "-"*40 + "\n")

Celltype: MCF-7
                Positives  Negatives
CTCF                21001      36689
TCF7L2              10297     405259
ZNF217               9945     404761
GATA3_(SC-268)       6083     415270

----------------------------------------

Celltype: GM12878
                     Positives  Negatives
CTCF                     40107     398210
YY1_(SC-281)             30998     349533
TBP                      14890     432228
Egr-1                    16324     424484
Mxi1_(AF4185)            17747     403510
SRF                       8546     468662
MAZ_(ab85725)            18972     395102
ELK1_(1277-1)             5584     483172
SIX5                      4843     489275
USF-1                     9779     475641
SP1                      18248     376855
RFX5_(200-401-194)        4340     504861
ELF1_(SC-631)            22998     374019
ATF2_(SC-81188)          23490     372754
NF-YB                    13305     474014
USF2                      9028     477902
Znf143_(16618-1-AP)     

In [5]:
# Find celltype-TF combinations where positives > negatives
more_positives = []
for celltype, stats in dicts.items():
    for tf, (pos, neg) in stats.items():
        if pos > neg:
            more_positives.append((celltype, tf, pos, neg))

more_positives

[('IMR90', 'MafK_(ab50322)', 40790, 0),
 ('HCT-116', 'CTCF', 50289, 46588),
 ('PANC-1', 'TCF7L2', 13382, 0),
 ('HEK293', 'CTCF', 47319, 12222)]

Of those HEK is the only one used in training I think?

In [ ]:
central = file["central"][:]
atac_track = central[0,:]
num_pos = (central[1,:]==1).sum()
num_overlap = ((central[1,:]==2) & (atac_track==1)).sum()
num_neg = ((central[1,:]==0) & (atac_track==1)).sum()

In [53]:
metrics_tables = {}

for h5t_file in h5t_files:
    file_path = os.path.join(data_folder, h5t_file)
    file = h5torch.File(file_path, 'r')
    prot_names = file["0/prot_names"][:]
    central = file["central"][:]
    celltype = h5t_file.split('.')[0]

    # Find ATAC index and track
    atac_idx = np.where(prot_names == b"ATAC_peak")[0]
    if len(atac_idx) == 0:
        continue
    atac_idx = atac_idx[0]
    atac_track = central[atac_idx, :]

    tf_metrics = {}
    for i, name in enumerate(prot_names):
        if i == atac_idx:
            continue  # skip ATAC
        tf_name = name.decode()
        tf_track = central[i,:]
        num_pos = (tf_track==1).sum()

        overlap_locs = (tf_track==2) & (atac_track==1)

        # Find indices where tf_track is 1 (true positions)
        true_indices = np.where(tf_track == 1)[0]

        # Shift overlap_locs left and right
        before = np.roll(overlap_locs, 1)
        after = np.roll(overlap_locs, -1)
        before[0] = False
        after[-1] = False

        # Find true_indices where either neighbor is True
        bordered_mask = before[true_indices] | after[true_indices]
        bordered_by_overlap = true_indices[bordered_mask]
        num_overlap = len(bordered_by_overlap)
        num_neg = ((tf_track==0) & (atac_track==1)).sum()


        tf_metrics[tf_name] = (int(num_pos), int(num_overlap), int(num_neg))

    metrics_tables[celltype] = pd.DataFrame(
        tf_metrics, index=['num_pos', 'num_overlap', 'num_neg']
    ).T

# Example: display the table for A549
# Add perc_pos_overlap column for all celltypes
for celltype, table in metrics_tables.items():
    table['perc_pos_overlap'] = (table['num_overlap'] / table['num_pos']) * 100

# Example: display the table for A549
metrics_tables['A549']

,num_pos,num_overlap,num_neg,perc_pos_overlap
CTCF,45735,35368,264180,77.332459
YY1_(SC-281),10277,4130,294811,40.186825
CREB1_(SC-240),15907,6432,289165,40.435029
Max,9891,3370,294663,34.071378
TCF12,20906,7148,286585,34.191141
FOSL2,28763,15696,279509,54.570107
ELF1_(SC-631),8618,2605,295893,30.227431
BHLHE40,3125,884,300892,28.288000
ATF3,6588,1396,297696,21.190043
USF-1,8448,2720,292009,32.196970


In [54]:
metrics_tables

{'MCF-7':                 num_pos  num_overlap  num_neg  perc_pos_overlap
 CTCF              21001        11938   237014         56.844912
 TCF7L2            10297         5670   301594         55.064582
 ZNF217             9945         5569   301356         55.997989
 GATA3_(SC-268)     6083         2634   305314         43.301003,
 'GM12878':                      num_pos  num_overlap  num_neg  perc_pos_overlap
 CTCF                   40107        23281   330257         58.047224
 YY1_(SC-281)           30998         8891   338762         28.682496
 TBP                    14890         3866   353154         25.963734
 Egr-1                  16324         4155   352255         25.453320
 Mxi1_(AF4185)          17747         3714   350763         20.927481
 SRF                     8546         1758   359070         20.571027
 MAZ_(ab85725)          18972         3686   349564         19.428632
 ELK1_(1277-1)           5584          670   361311         11.998567
 SIX5                   

Hmm so this overlap is surprisingly low.... Problem?


In [51]:
metrics_tables = {}

for h5t_file in h5t_files:
    file_path = os.path.join(data_folder, h5t_file)
    file = h5torch.File(file_path, 'r')
    prot_names = file["0/prot_names"][:]
    central = file["central"][:]
    celltype = h5t_file.split('.')[0]

    # Find ATAC index and track
    atac_idx = np.where(prot_names == b"ATAC_peak")[0]
    if len(atac_idx) == 0:
        continue
    atac_idx = atac_idx[0]
    atac_track = central[atac_idx, :]

    tf_metrics = {}
    for i, name in enumerate(prot_names):
        if i == atac_idx:
            continue  # skip ATAC
        tf_name = name.decode()
        tf_track = central[i,:]
        num_pos = (tf_track==1).sum()
        num_overlap = ((tf_track==2) & (atac_track==1)).sum()
        num_neg = ((tf_track==0) & (atac_track==1)).sum()


        tf_metrics[tf_name] = (int(num_pos), int(num_overlap), int(num_neg))

    metrics_tables[celltype] = pd.DataFrame(
        tf_metrics, index=['num_pos', 'num_overlap', 'num_neg']
    ).T

# Example: display the table for A549
# Add perc_pos_overlap column for all celltypes
for celltype, table in metrics_tables.items():
    table['perc_pos_overlap'] = (table['num_overlap'] / table['num_pos']) * 100

# Example: display the table for A549
metrics_tables['A549']

,num_pos,num_overlap,num_neg,perc_pos_overlap
CTCF,45735,39642,264180,86.677599
YY1_(SC-281),10277,9011,294811,87.681230
CREB1_(SC-240),15907,14657,289165,92.141824
Max,9891,9159,294663,92.599333
TCF12,20906,17237,286585,82.450014
FOSL2,28763,24313,279509,84.528735
ELF1_(SC-631),8618,7929,295893,92.005106
BHLHE40,3125,2930,300892,93.760000
ATF3,6588,6126,297696,92.987250
USF-1,8448,11813,292009,139.831913


Yeah I think it can be more then 100% because sometimes one peak overlaps with two atac seq sequences. So maybe not the best way to do it.